In [1]:
from mimicfouretl.data_insights import create_ui
from mimicfouretl.feature_engineering import *
import mimicfouretl.bigquery_utils as bq

In [2]:
create_ui()

Dropdown(description='Dataset:', options=('hosp/pharmacy', 'hosp/provider', 'hosp/poe_detail', 'hosp/admission…

Output()

In [3]:
bq.set_credentials_file('../bq_credentials/client_secret.json')
bq.set_project_id('micro-vine-412020')
client = bq.get_client(use_service_account_auth=True)
spark = bq.get_spark_session(client, use_service_account_auth=True)

24/03/24 16:12:37 WARN Utils: Your hostname, Dunyas-Laptop.local resolves to a loopback address: 127.0.0.1; using 10.0.0.117 instead (on interface en0)
24/03/24 16:12:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/dunyaoguz/.ivy2/cache
The jars for the packages stored in: /Users/dunyaoguz/.ivy2/jars
com.google.cloud.spark#spark-bigquery-with-dependencies_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-1f3ef7fd-45a0-46fd-ad5c-3602da2ccc1f;1.0
	confs: [default]


:: loading settings :: url = jar:file:/Users/dunyaoguz/anaconda3/envs/mimic-iv-etl/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.37.0 in central
	[0.37.0] com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;latest.version
:: resolution report :: resolve 1019ms :: artifacts dl 2ms
	:: modules in use:
	com.google.cloud.spark#spark-bigquery-with-dependencies_2.12;0.37.0 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   1   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-1f3ef7fd-45a0-46fd-ad5c-3602da2ccc1f
	confs: [default]
	0 artifacts copied, 1 already retrieved (0kB/3ms)
24/03/24 16:12:39 WARN NativeCodeLoader: Unable to load native-hadoo

In [ ]:
results = get_item_frequency(spark, column_name='caregiver_id', dataset='mimic_iv.icu_outputevents')
bq.display_sampled_df(results)

In [ ]:
results = get_temporal_trends(spark=spark, 
                              item_id='226559', 
                              column_name='itemid', 
                              date_column='charttime', 
                              dataset='mimic_iv.icu_outputevents',
                              limit=10)
bq.display_sampled_df(results)

In [ ]:
results = get_outcomes_by_item(spark=spark, 
                              item_id='227074', 
                              item_column='itemid',
                              item_dataset='mimic_iv.icu_inputevents',
                              outcome_column='value',
                              outcome_dataset='mimic_iv.icu_outputevents')
bq.display_sampled_df(results)

In [ ]:
results = get_abnormal_item_analysis(spark=spark, 
                              item_id=51200, 
                              item_column='itemid',
                              value_column='value',
                              bounds={'lower': '0.5'},
                              dataset='mimic_iv.hosp_labevents')
bq.display_sampled_df(results)

In [ ]:
results = get_provider_activity_analysis(spark=spark, 
                                         provider_id='P60CC5', 
                                         dataset_columns={'mimic_iv.hosp_microbiologyevents':
                                                                {'provider': 'order_provider_id', 
                                                                 'activity':'spec_type_desc'}})
bq.display_sampled_df(results)

In [ ]:
results = get_co_occurrence_analysis(spark=spark, 
                                     dataset='mimic_iv.hosp_labevents',
                                     primary_column='priority',
                                     secondary_column='comments')
bq.display_sampled_df(results)

In [4]:
results = calculate_event_to_death_interval(spark=spark, 
                                     event_date_column='charttime',
                                     event_dataset='mimic_iv.hosp_labevents')
bq.display_sampled_df(results)

GENERATED QUERY:
 
    WITH death_dates AS (
        SELECT subject_id, dod AS date_of_death
        FROM `micro-vine-412020.mimic_iv.hosp_patients`
        WHERE dod IS NOT NULL
    ),
    events AS (
        SELECT subject_id, hadm_id, charttime AS event_date
        FROM `mimic_iv.hosp_labevents`
    )
    SELECT e.subject_id, e.hadm_id, e.event_date, d.date_of_death,
           DATEDIFF(d.date_of_death, e.event_date) AS days_to_death
    FROM events e
    JOIN death_dates d ON e.subject_id = d.subject_id
    


Py4JJavaError: An error occurred while calling o36.load.
: com.google.cloud.bigquery.connector.common.BigQueryConnectorException: Error creating destination table using the following query: [WITH death_dates AS (
        SELECT subject_id, dod AS date_of_death
        FROM `micro-vine-412020.mimic_iv.hosp_patients`
        WHERE dod IS NOT NULL
    ),
    events AS (
        SELECT subject_id, hadm_id, charttime AS event_date
        FROM `mimic_iv.hosp_labevents`
    )
    SELECT e.subject_id, e.hadm_id, e.event_date, d.date_of_death,
           DATEDIFF(d.date_of_death, e.event_date) AS days_to_death
    FROM events e
    JOIN death_dates d ON e.subject_id = d.subject_id]
	at com.google.cloud.bigquery.connector.common.BigQueryClient.materializeTable(BigQueryClient.java:695)
	at com.google.cloud.bigquery.connector.common.BigQueryClient.materializeQueryToTable(BigQueryClient.java:617)
	at com.google.cloud.bigquery.connector.common.BigQueryClient.getReadTable(BigQueryClient.java:400)
	a